<a href="https://colab.research.google.com/github/Kedyi/Sentiment_analysis/blob/master/Sentiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
#get api details
consumerKey = ''
consumerSecret = ''
accessToken = ''
accessTokenSecret = ''


In [ ]:
authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret)
authenticate.set_access_token(accessToken, accessTokenSecret)

#api obj
api = tweepy.API(authenticate, wait_on_rate_limit=True)

In [ ]:
#extracting tweets
posts=api.user_timeline(screen_name="BillGates", count=100, lang = "en", tweet_mode = "extended")

In [ ]:
df = pd.DataFrame([tweet.full_text for tweet in posts], columns=['Tweets'])

#Show the first 5 rows of data
df.head()

In [ ]:
#data cleaning
def cleanTweets(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text) #removes @mentions
  text = re.sub(r'#', '', text) # remove #
  text = re.sub(r'RT[\s]+', '', text) #remove retweets
  text = re.sub(r'https?:\/\/\S+', '', text)#Removes hyperlink

  return text

df['Tweets']=df['Tweets'].apply(cleanTweets)

#show
df



In [ ]:
#Create a function to get subjectivity
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

#get polarity
def getPolarity(text):
  return TextBlob(text).sentiment.polarity

#create two new columns
df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Tweets'].apply(getPolarity)

#show new dataframe with new column
df

In [ ]:
# plot word cloud
allWords = ''.join([twts for twts in df['Tweets']])
wordCloud =  WordCloud(width = 500, height=300, random_state =21, max_font_size = 119).generate(allWords)

plt.imshow(wordCloud, interpolation = "bilinear")
plt.axis('off')
plt.show()

In [ ]:
#create a function to compute the negative, neutral and positve analysis
def getAnalysis(score):
  if score < 0:
    return 'Negative'
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positive'

df['Analysis'] = df['Polarity'].apply(getAnalysis)

#show dataframe
df

In [ ]:
#print all the positive tweets
j=1
sortedDF = df.sort_values(by=['Polarity'])
for i in range(0, sortedDF.shape[0]):
  if(sortedDF['Analysis'][i]=='Positive'):
    print(str(j) + ')' + sortedDF['Tweets'][i])
    print()
    j=j+1

In [ ]:
#print negative tweets
j=1
sortedDF = df.sort_values(by=['Polarity'], ascending='False')
for i in range(0, sortedDF.shape[0]):
  if(sortedDF['Analysis'][i]=='Negative'):
     print(str(j) + ')' + sortedDF['Tweets'][i])
     print()
     j=j+1
     

In [ ]:
#Plot polarity and subjectivity
plt.figure(figsize=(8,6))
for i in range(0, df.shape[0]):
  plt.scatter(df['Polarity'][i], df['Subjectivity'], color = 'Blue')

plt.title('Sentiment Analysis')
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.show()

In [ ]:
#get the percentage of positve tweets
ptweets = df[df.Analysis =='Positive']
ptweets = ptweets['Tweets']

round((ptweets.shape[0] / df.shape[0])*100, 1)

In [ ]:
#get percentage of native tweets
ntweets = df[df.Analysis =='Negative']
ntweets = ntweets['Tweets']

round((ntweets.shape[0] / df.shape[0])*100, 1)

In [ ]:
#show value count
df['Analysis'].value_counts()

#plot and visualize the counts
plt.title('Sentiment analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
df['Analysis'].value_counts().plot(kind='bar')
plt.show()